In [13]:
from lark import Lark


In [200]:
subprogram_body = Lark(r"""
    procedure_body : procedure_specification contract # "is" declarative_part statements "end" NAME

    procedure_specification : "procedure" procedure_name ["(" parameter_list ")"] 
    parameter_list : parameter [";" parameter]*
    parameter      : variable_name ":"  mode type_name
    contract       : classic_subprogram_contract
                   | spark_2014_subprogram_contract

    classic_subprogram_contract : [global_definition] [dependency_relation] [precondition] [postcondition]
    
    global_definition : "global" mode global_item_list ";" [mode global_item_list ";"]*
    global_item_list  : global_item ["," global_item]*

    dependency_relation : "derives" dependency_clauses
    dependency_clauses : dependency_clause ["&" dependency_clause]* ["&" null_dependency_clause]";"
    dependency_clause : exported_variable_list "from" [extended_imported_item_list]
                      | null_dependency_clause
    null_dependency_clause : NULL "from" imported_item_list

    exported_variable_list : exported_variable ["," exported_variable]*
    extended_imported_item_list : STAR | [STAR ","] imported_item_list
    imported_item_list : imported_item ["," imported_item]*

    precondition : "pre" classic_predicate
    postcondition : "post" classic_predicate

    classic_predicate : STRING

    spark_2014_subprogram_contract : ["with" aspect_clauses]
    aspect_clauses : aspect_clause ["," aspect_clause]*
    
    aspect_clause : [global_aspect] [dependency_aspect] [precondition_aspect] [postcondition_aspect]

    global_aspect : "global" "=>" global_specification
    global_specification : "(" moded_global_list ["," moded_global_list]* ")"
                         | global_list
                         | null_global_specification
    moded_global_list : mode_selector "=>" global_list
    global_list : global_item 
                | "(" global_item ["," global_item]* ")"
    mode_selector     : INPUT | OUTPUT | IN_OUT | PROOF_IN
    null_global_specification : NULL

    dependency_aspect : "depends" "=>" dependency_list
    dependency_list : NULL
                    | "(" dependency_specification ["," dependency_specification]* ")"
    dependency_specification : output_list "=>" [PLUS] input_list
                             | null_dependecy_specification
    null_dependecy_specification : NULL "=>" input_list
    output_list : output
                | "(" output ["," output]*")"
    input_list : input
               | "(" input ["," input]* ")"
               | NULL

    precondition_aspect : "pre" "=>" predicate
    postcondition_aspect : "post" "=>" predicate

    predicate : STRING
            
   mode : [IN] [OUT]

    exported_variable : NAME
    global_item       : NAME
    imported_item     : NAME
    input             : NAME
    output            : NAME
    procedure_name    : NAME
    type_name         : NAME
    variable_name     : NAME

    GLOBAL   : "global"
    IN       : "in"
    IN_OUT   : "in_out"
    INPUT    : "input"
    NULL     : "null"
    OUT      : "out"
    OUTPUT   : "output"
    PLUS     : "+"
    PROOF_IN : "proof_in"
    STAR     : "*"
    
    %import common.CNAME -> NAME
    %import common.ESCAPED_STRING -> STRING
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

    ANNO : "--#"
    %ignore ANNO

    """, start='procedure_body', lexer='basic')


In [201]:
text='''procedure P1 (a : in integer; b : in out integer; c : natural)'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [None, None, None, None])])])

In [202]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    classic_subprogram_contract
      None
      None
      None
      None



In [203]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [Tree(Token('RULE', 'global_definition'), [Tree(Token('RULE', 'mode'), [Token('I

In [204]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    classic_subprogram_contract
      global_definition
        mode
          in
          None
        global_item_list
          global_item	g
          global_item	h
          global_item	j
        mode
          in
          out
        global_item_list
          global_item	r
          global_item	s
          global_item	t
        mode
          None
          out
        global_item_list
          global_item	x
          global_item	y
          global_item	z
      None
      None
      None



In [205]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          with global => (input => (g, h, j), 
                     in_out => (r, s, j), 
                     output => (x, y, z))'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          with global => (input => (g, h, j), 
                     in_out => (r, s, j), 
                     output => (x, y, z))


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'spark_2014_subprogram_contract'), [Tree(Token('RULE', 'aspect_clauses'), [Tree(Token('RULE', 'aspect_clause'), 

In [206]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    spark_2014_subprogram_contract
      aspect_clauses
        aspect_clause
          global_aspect
            global_specification
              moded_global_list
                mode_selector	input
                global_list
                  global_item	g
                  global_item	h
                  global_item	j
              moded_global_list
                mode_selector	in_out
                global_list
                  global_item	r
                  global_item	s
                  global_item	j
              moded_global_list
                mode_selector	out

In [207]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [Tree(Token('RULE', 'global_definition'), [Tree(Token('RULE', 'mode'), [Token('I

In [208]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    classic_subprogram_contract
      global_definition
        mode
          in
          None
        global_item_list
          global_item	g
          global_item	h
          global_item	j
        mode
          in
          out
        global_item_list
          global_item	r
          global_item	s
          global_item	t
        mode
          None
          out
        global_item_list
          global_item	x
          global_item	y
          global_item	z
      dependency_relation
        dependency_clauses
          dependency_clause
            exported_variable_list

In [209]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z))'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z))


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'spark_2014_subprogram_contract'), [Tree(Token('RULE', 'aspect_clauses'), [Tree(Token('RULE', 'aspect_clause'), 

In [210]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    spark_2014_subprogram_contract
      aspect_clauses
        aspect_clause
          global_aspect
            global_specification
              moded_global_list
                mode_selector	input
                global_list
                  global_item	g
                  global_item	h
                  global_item	j
              moded_global_list
                mode_selector	in_out
                global_list
                  global_item	r
                  global_item	s
                  global_item	t
              moded_global_list
                mode_selector	out

In [211]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;
          --# pre "g > 0"
          --# post "x = g + h"'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          --# global in g, h, j;
          --#        in out r, s, t;
          --#        out    x, y, z;
          --# derives r, x, y from g, h, s &
                      s from *             &
                      t from *, y, z;
          --# pre "g > 0"
          --# post "x = g + h"


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'classic_subprogram_contract'), [Tree(Token('RULE', 'global_definition'), [Tree(Token('RULE', 'mode'), [Token('I

In [212]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    classic_subprogram_contract
      global_definition
        mode
          in
          None
        global_item_list
          global_item	g
          global_item	h
          global_item	j
        mode
          in
          out
        global_item_list
          global_item	r
          global_item	s
          global_item	t
        mode
          None
          out
        global_item_list
          global_item	x
          global_item	y
          global_item	z
      dependency_relation
        dependency_clauses
          dependency_clause
            exported_variable_list

In [213]:
text = '''procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z)),
             pre     => "g > 0",
             post    => "x = g + h"'''
print (text)
subprogram_body.parse(text)

procedure P1 (a : in integer; b : in out integer; c : natural)
          with
             global => (input => (g, h, j),
                       in_out => (r, s, t),
                       output =>  (x, y, z)),
             depends => ((r, x, y) => (g, h, s),
                         s => + null,
                         t => + (y, z)),
             pre     => "g > 0",
             post    => "x = g + h"


Tree(Token('RULE', 'procedure_body'), [Tree(Token('RULE', 'procedure_specification'), [Tree(Token('RULE', 'procedure_name'), [Token('NAME', 'P1')]), Tree(Token('RULE', 'parameter_list'), [Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'a')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'b')]), Tree(Token('RULE', 'mode'), [Token('IN', 'in'), Token('OUT', 'out')]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'integer')])]), Tree(Token('RULE', 'parameter'), [Tree(Token('RULE', 'variable_name'), [Token('NAME', 'c')]), Tree(Token('RULE', 'mode'), [None, None]), Tree(Token('RULE', 'type_name'), [Token('NAME', 'natural')])])])]), Tree(Token('RULE', 'contract'), [Tree(Token('RULE', 'spark_2014_subprogram_contract'), [Tree(Token('RULE', 'aspect_clauses'), [Tree(Token('RULE', 'aspect_clause'), 

In [214]:
print( _.pretty() )

procedure_body
  procedure_specification
    procedure_name	P1
    parameter_list
      parameter
        variable_name	a
        mode
          in
          None
        type_name	integer
      parameter
        variable_name	b
        mode
          in
          out
        type_name	integer
      parameter
        variable_name	c
        mode
          None
          None
        type_name	natural
  contract
    spark_2014_subprogram_contract
      aspect_clauses
        aspect_clause
          global_aspect
            global_specification
              moded_global_list
                mode_selector	input
                global_list
                  global_item	g
                  global_item	h
                  global_item	j
              moded_global_list
                mode_selector	in_out
                global_list
                  global_item	r
                  global_item	s
                  global_item	t
              moded_global_list
                mode_selector	out